# Large Model Support: Keras Introduction

Additional Resourcers:
- https://www.ibm.com/support/knowledgecenter/en/SS5SF7_1.6.0/navigation/pai_getstarted_tflmsv2.html
- https://github.com/IBM/tensorflow-large-model-support
- https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
- https://stackoverflow.com/questions/46493419/use-a-generator-for-keras-model-fit-generator

## Downloading necessary dependencies

In [0]:
! git clone https://github.com/IBM/tensorflow-large-model-support.git
! pip install ./tensorflow-large-model-support

fatal: destination path 'tensorflow-large-model-support' already exists and is not an empty directory.
Processing ./tensorflow-large-model-support
  Stored in directory: /root/.cache/pip/wheels/00/30/9f/af0877bd7f5dab704d0eacfd5a954c020d3783fc9b07e23542
Successfully built tensorflow-large-model-support
  Found existing installation: tensorflow-large-model-support 0.1.0
    Uninstalling tensorflow-large-model-support-0.1.0:
      Successfully uninstalled tensorflow-large-model-support-0.1.0


In [0]:
! pip install memory_profiler

## Preprocessing

In [0]:
from tensorflow_large_model_support import LMSKerasCallback
import numpy as np 
import pandas as pd 
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution1D, Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [0]:
%load_ext memory_profiler 
# Used to examining memory usage

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [0]:
# LMSKerasCallback and LMS share a set of keyword arguments. Here we just
# use the default options.
lms_callback = LMSKerasCallback()

In [0]:
# Creating a linearly separable dataset using Gaussian Distributions.
# The first half of the number in Y is 0 and the other half 1.
# Therefore I made the first half of the two features quite different from
# the second half of the features (setting the value of the means quite 
# similar) so that make quite simple the classification between the 
# classes (the data is linearly separable).
dataset_len = 2000000
dlen = int(dataset_len/2)
X_11 = pd.Series(np.random.normal(2,2,dlen))
X_12 = pd.Series(np.random.normal(9,2,dlen))
X_1 = pd.concat([X_11, X_12]).reset_index(drop=True)
X_21 = pd.Series(np.random.normal(1,3,dlen))
X_22 = pd.Series(np.random.normal(7,3,dlen))
X_2 = pd.concat([X_21, X_22]).reset_index(drop=True)
X_31 = pd.Series(np.random.normal(3,1,dlen))
X_32 = pd.Series(np.random.normal(3,4,dlen))
X_3 = pd.concat([X_31, X_32]).reset_index(drop=True)
Y = pd.Series(np.repeat([0,1],dlen))
df = pd.concat([X_1, X_2, X_3, Y], axis=1)
df.columns = ['X1', 'X2', 'X3', 'Y']
df.head()

,X1,X2,X3,Y
0,0.975025,3.634938,3.784286,0
1,0.901440,4.542160,2.406820,0
2,0.954707,2.570255,1.763635,0
3,6.044522,1.149320,2.940318,0
4,-0.754139,6.806385,3.548091,0


In [0]:
X = df.drop(['Y'], axis = 1).values
y = df['Y']

In [0]:
def preproces(df, X, y, train_size = 0.80):
  # label_encoder object knows how to understand word labels. 
  label_encoder = preprocessing.LabelEncoder() 

  # Encode labels
  y = label_encoder.fit_transform(y) 

  # identify shape and indices
  num_rows, num_columns = df.shape
  delim_index = int(num_rows * train_size)

  # Splitting the dataset in training and test sets
  X_train, y_train = X[:delim_index, :], y[:delim_index]
  X_test, y_test = X[delim_index:, :], y[delim_index:]

  # Checking sets dimensions
  print('X_train dimensions: ', X_train.shape, 'y_train: ', y_train.shape)
  print('X_test dimensions:', X_test.shape, 'y_validation: ', y_test.shape)

  # Checking dimensions in percentages
  total = X_train.shape[0] + X_test.shape[0]
  print('X_train Percentage:', (X_train.shape[0]/total)*100, '%')
  print('X_test Percentage:', (X_test.shape[0]/total)*100, '%')
  
  return X_train, y_train, X_test, y_test

In [0]:
X_train, y_train, X_test, y_test = preproces(df, X, y)

X_train dimensions:  (1600000, 3) y_train:  (1600000,)
X_test dimensions: (400000, 3) y_validation:  (400000,)
X_train Percentage: 80.0 %
X_test Percentage: 20.0 %


## Machine Learning

In [0]:
model = Sequential()
model.add(Dense(12, activation='relu', input_dim=df.shape[1]-1))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

batch_size = 8

### Large Model Support

In [0]:
def generator(X_data, y_data, batch_size):

  samples_per_epoch = X_data.shape[0]
  number_of_batches = samples_per_epoch/batch_size
  counter=0

  while 1:

    X_batch = np.array(X_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
    y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
    counter += 1
    yield X_batch,y_batch

    #restart counter to yeild data in the next epoch as well
    if counter >= number_of_batches:
        counter = 0

In [0]:
%%memit

model.fit_generator(generator(X_train, y_train,batch_size), epochs=2,steps_per_epoch = X_train.shape[0]/batch_size, callbacks=[lms_callback])

# Peak Memory = Memory usage of the Python interpreter after that line has been executed. Increment = represents the difference in memory of the current line with respect to the last one. 

Epoch 1/2
200000/200000 [==============================] - 601s 3ms/step - loss: 0.0222 - acc: 0.9984
Epoch 2/2
200000/200000 [==============================] - 596s 3ms/step - loss: 0.0203 - acc: 0.9984
peak memory: 2834.80 MiB, increment: 2.88 MiB


In [0]:
LMS_preds = model.predict_classes(X_test)

In [0]:
LMS_acc = accuracy_score(y_test, LMS_preds)
print("Model accuracy using Large Model Support:", LMS_acc*100, '%')

Model accuracy using Large Model Support: 99.9995 %


### Keras

In [0]:
%%memit
#without generator
model.fit(x = np.array(X_train), y = np.array(y_train), batch_size = batch_size, epochs = 2)

Epoch 1/2
1600000/1600000 [==============================] - 537s 336us/step - loss: 0.0449 - acc: 0.9846
Epoch 2/2
1600000/1600000 [==============================] - 538s 336us/step - loss: 0.0403 - acc: 0.9857
peak memory: 2862.26 MiB, increment: 26.15 MiB


In [0]:
preds = model.predict_classes(X_test)

In [0]:
sk_pred = np.round(preds)
sk_acc = accuracy_score(y_test, sk_pred)
print("Model accuracy using Sklearn:", sk_acc*100, '%')

Model accuracy using Sklearn: 98.4795 %
